In [ ]:
pip install rembg colabcode ultralytics

In [ ]:
from colabcode import ColabCode
ColabCode(port = 10000, authtoken = "2POStfHNN5t4P7dmKr3RqgrKpID_2KhfHwQ1ZceUfLtFFUX7h")

In [ ]:
from rembg import remove
from pathlib import Path
from cv2 import imread, threshold, findContours, RETR_CCOMP, THRESH_BINARY, CHAIN_APPROX_NONE

for path in Path(".").glob("./datasets/images/*.*"):

    image = imread(str(path))
    alpha = remove(image)[:, :, 3]

    mask = threshold(alpha, 150, 255, THRESH_BINARY)[1]
    contours = findContours(mask, RETR_CCOMP, CHAIN_APPROX_NONE)[0]
    
    segmentation = []
    for item in contours:
        contour = item.flatten().tolist()
        contour = [(contour[j], contour[j + 1]) for j in range(0, len(contour), 2)]
        contour = [(x / image.shape[1], y / image.shape[0]) for x, y in contour]
        segmentation += contour

    with open(f"./datasets/labels/{path.stem}.txt", "w") as annotations:
        annotations.write(f"0 {' '.join([f'{x:.5f} {y:.5f}' for x, y in segmentation])}")

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")
model.train(data = "data.yaml")

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/segment/train/weights/best.pt")
results = model("https://cdn.mos.cms.futurecdn.net/mhJ2yWNwMtNcmijZqVEDDW.jpg")